In [29]:
import os
import sys
import codecs
import collections

In [30]:
import nltk
from lxml import etree
from lxml.html.clean import Cleaner
from pymystem3 import Mystem

In [31]:
file = codecs.open('datasets/golos_io_topics.xml', encoding='utf-8')
data = file.read();

In [32]:
cleaner = Cleaner()
cleaner.allow_tags = ['item', 'field']
cleaner.remove_unknown_tags = False
clean_data = cleaner.clean_html(data)

In [33]:
fout = codecs.open('datasets/golos_io_topics-res.xml', encoding='utf-8', mode='w+')
fout.write(clean_data)
fout.close()

In [34]:
# формируем список стоп-слов и добавляем свои
from nltk.corpus import stopwords
stopwords_list = stopwords.words('russian')
more_stopwords = '..., без, более, бы, был, была, были, было, быть, вам, вас, ведь, весь, вдоль, вместо, вне, вниз, внизу, внутри, во, вокруг, вот, все, всегда, всего, всех, вы, где, да, давай, давать, даже, для, до, достаточно, его, ее, её, если, есть, ещё, же, за, здесь, из, из-за, или, им, иметь, их, как, как-то, кто, когда, кроме, кто, ли, либо, мне, может, мои, мой, мы, на, навсегда, над, надо, наш, не, него, неё, нет, ни, них, но, ну, об, однако, он, она, они, оно, от, отчего, очень, по, под, после, потому, потому что, почти, при, про, снова, со, так, также, такие, такой, там, те, тем, то, того, тоже, той, только, том, тут, туда, ты, уже, хотя, чего, чего-то, чей, чем, что, чтобы, чьё, чья, эта, эти, это'
more_stopwords_list = more_stopwords.split(", ")
stopwords_list += more_stopwords_list

In [35]:
tree = etree.fromstring(clean_data)
items = tree.xpath("//field[@name='body']")

In [117]:
seq_list = [item.text if item.text is not None else " " for item in items]

In [118]:
# delete all URLs in text
import re
url_pattern = re.compile(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')

seq_list = [url_pattern.sub("", item) for item in seq_list]

In [119]:
fout = codecs.open('datasets/golos_io_topics-res2.txt', encoding='utf-8', mode='w+')
for item in seq_list:
    print(item, file=fout, sep="\n")
fout.close()

In [120]:
mystem = Mystem()
def lemmatize_words(word_list):
    processed_word_list = []
    for word in word_list:
        word = mystem.lemmatize(word)[0]
        processed_word_list.append(word)
    return processed_word_list    

In [121]:
def remove_stopwords(word_list):
        processed_word_list = []      
        for word in word_list:
            word = word.lower() # in case they arenet all lower cased
            if word not in stopwords_list and len(word) > 2:
                processed_word_list.append(word)
        return processed_word_list    

In [122]:
test_tokenized = nltk.word_tokenize(seq_list[1100])
test_clean = remove_stopwords(test_tokenized)

In [123]:
print(test_clean)

['всем', 'привет', 'недавно', 'одна', 'девочка', 'звала', 'crp.center', 'освободилась', '1200', 'долларов', 'думаю', 'эххх', 'вложила', 'криптовалюту', 'верила', 'проект', 'знаю', 'почему', 'слава', 'богу', 'интуиция', 'подвела', 'говорит', '1000', 'долларов', 'сняла', '000', 'слава', 'богу', 'представьте', 'сколько', 'людей', 'потеряло', 'печально', 'видео', 'обращение', 'виталий', 'говорит', 'будут', 'работать', 'обычно', 'сказки', 'вкладывайте', 'всем', 'мира', 'стабильного', 'дохода']


In [124]:
def preprocess_doc(doc_text):
    words_list = remove_stopwords(nltk.word_tokenize(doc_text))
# без леммизации

    return " ".join(words_list)

In [125]:
def preprocess_lemma_doc(doc_text):
    words_list = remove_stopwords(nltk.word_tokenize(doc_text))
    lemmas_list = lemmatize_words(words_list)
    lemmas_list = remove_stopwords(lemmas_list)
    return " ".join(lemmas_list)

In [126]:
lemmas_doc = preprocess_doc(seq_list[1100])
lemmas_doc

'всем привет недавно одна девочка звала crp.center освободилась 1200 долларов думаю эххх вложила криптовалюту верила проект знаю почему слава богу интуиция подвела говорит 1000 долларов сняла 000 слава богу представьте сколько людей потеряло печально видео обращение виталий говорит будут работать обычно сказки вкладывайте всем мира стабильного дохода'

In [127]:
lemmas_doc = preprocess_lemma_doc(seq_list[1100])
lemmas_doc

'привет недавно девочка звать crp освобождаться 1200 доллар думать эххх вкладывать криптовалюта верить проект знать почему слава бог интуиция подводить говорить 1000 доллар снимать 000 слава бог представлять сколько человек потерять печально видео обращение виталий говорить работать обычно сказка вкладывать мир стабильный доход'

In [128]:
fout = codecs.open('datasets/golos_io_topics-res-lines.txt', encoding='utf-8', mode='w+')
for item in seq_list:
    if item is not None:
        processed_doc = preprocess_doc(item)
        print(processed_doc, file=fout, sep="\n")
fout.close()

In [129]:
fout = codecs.open('datasets/golos_io_topics-res-lemmas.txt', encoding='utf-8', mode='w+')
for item in seq_list:
    if item is not None:
        processed_lemma_doc = preprocess_lemma_doc(item)
        print(processed_lemma_doc, file=fout, sep="\n")
fout.close()